In [1]:
import cv2
import numpy as np
import random as rng

img = cv2.imread('IMG_2960.jpg')

In [2]:
def scale_img(img, scale_width=1000):
    scale_percent = (scale_width * 100) / img.shape[1]
    height = int(img.shape[0] * scale_percent / 100)
    dim = (scale_width, height)
    return cv2.resize(img, dim)

In [3]:
dst = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
cv2.imshow('Binary image',scale_img(dst))
cv2.waitKey(0)

-1

In [4]:
gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY) # convert to grayscale
# blur = cv2.blur(gray, (3, 3)) # blur the image
# ret, thresh = cv2.threshold(blur, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
# thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
blur = cv2.GaussianBlur(gray,(5,5),0)
ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


In [5]:
drawing = cv2.drawContours(img, contours, -1, (0,255,0), 3)
cv2.imshow('Binary image',scale_img(drawing))
cv2.waitKey(0)

-1

In [31]:
#Noice Removal
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
blur = cv2.GaussianBlur(gray,(5,5),0)
ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
erodeImage = cv2.erode(thresh,kernel, iterations=1)
dilatedImg = cv2.dilate(erodeImage,kernel, iterations=1)
contours, hierarchy = cv2.findContours(dilatedImg,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [32]:
drawing = cv2.drawContours(img, contours, -1, (0,255,0), 3)
cv2.imshow('Binary image',scale_img(drawing))
cv2.waitKey(0)

-1

In [6]:
print(len(contours))


1528


In [24]:
def calculate_contour_distance(contour1, contour2):
    x1, y1, w1, h1 = cv2.boundingRect(contour1)
    c_x1 = x1 + w1/2
    c_y1 = y1 + h1/2

    x2, y2, w2, h2 = cv2.boundingRect(contour2)
    c_x2 = x2 + w2/2
    c_y2 = y2 + h2/2

    return max(abs(c_x1 - c_x2) - (w1 + w2)/2, abs(c_y1 - c_y2) - (h1 + h2)/2)

def merge_contours(contour1, contour2):
    return np.concatenate((contour1, contour2), axis=0)

def agglomerative_cluster(contours, threshold_distance=40.0):
    current_contours = list(contours)
    while len(current_contours) > 1:
        min_distance = None
        min_coordinate = None

        for x in range(len(current_contours)-1):
            for y in range(x+1, len(current_contours)):
                distance = calculate_contour_distance(current_contours[x], current_contours[y])
                if min_distance is None:
                    min_distance = distance
                    min_coordinate = (x, y)
                elif distance < min_distance:
                    min_distance = distance
                    min_coordinate = (x, y)

        if min_distance < threshold_distance:
            index1, index2 = min_coordinate
            current_contours[index1] = merge_contours(current_contours[index1], current_contours[index2])
            del current_contours[index2]
        else:
            break

    return current_contours

contours = agglomerative_cluster(contours)
print(len(contours))

1758


KeyboardInterrupt: 

In [13]:
# create hull array for convex hull points
hull = []

drawing = np.zeros((thresh.shape[0], thresh.shape[1], 3), np.uint8)
height, width, channels = img.shape

smalest_contour_area = height * width * 0.02 #contours smaler than 20% of the whole image should be removed

final_contours = []
# calculate points for each contour
for i in range(len(contours)):
    # creating convex hull object for each contour
    contour = contours[i]
    area = cv2.contourArea(contour)
    if area >= smalest_contour_area:
        # color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        rect = cv2.minAreaRect(contour)
        final_contours.append(rect)
        box = cv2.boxPoints(rect)
        box = np.intp(box)
        # cv2.drawContours(drawing, [box], 0, color)
        cv2.drawContours(drawing, [box], 0, 255)



cv2.imshow('Binary image',scale_img(drawing))
cv2.waitKey(0)
# final_contours

-1

In [10]:
drawing2 = cv2.drawContours(img, final_contours, -1, (0,255,0), 3)
cv2.imshow('Binary image',scale_img(drawing2))
cv2.waitKey(0)

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'drawContours'
> Overload resolution failed:
>  - Can't parse 'contours'. Sequence item with index 0 has a wrong type
>  - Can't parse 'contours'. Sequence item with index 0 has a wrong type


In [ ]:
    hull = cv2.convexHull(contour)
    phull = Polygon([[x, y] for (x, y) in
                     zip(hull[:, :, 0], hull[:, :, 1])])

def characterize_card_contour(card_contour,
                              max_segment_area,
                              image_area):
    """
    Calculates a bounding polygon for a contour, in addition
    to several charasteristic parameters.
    """
    phull = convex_hull_polygon(card_contour)
    if (phull.area < 0.1 * max_segment_area or
            phull.area < image_area / 1000.):
        # break after card size range has been explored
        continue_segmentation = False
        is_card_candidate = False
        bounding_poly = None
        crop_factor = 1.
    else:
        continue_segmentation = True
        bounding_poly = get_bounding_quad(phull)
        qc_diff = quad_corner_diff(phull, bounding_poly)
        crop_factor = min(1., (1. - qc_diff * 22. / 100.))
        is_card_candidate = bool(
            0.1 * max_segment_area < bounding_poly.area <
            image_area * 0.99 and
            qc_diff < 0.35 and
            0.25 < polygon_form_factor(bounding_poly) < 0.33)

    return (continue_segmentation,
            is_card_candidate,
            bounding_poly,
            crop_factor)

In [ ]:
# create an empty black image
drawing = np.zeros((thresh.shape[0], thresh.shape[1], 3), np.uint8)

# draw contours and hull points
for i in range(len(contours)):
    # color_contours = (0, 255, 0) # green - color for contours
    color = (255, 0, 0) # blue - color for convex hull
    # draw ith contour
    # cv2.drawContours(drawing, contours, i, color_contours, 1, 8, hierarchy)
    # draw ith convex hull object
    cv2.drawContours(drawing, hull, i, color, 1, 8)

In [ ]:
cv2.imshow('Binary image', scale_img(drawing))
cv2.waitKey(0)